# StarRocks | StarRocks> [StarRocks](https://www.starrocks.io/)は、高性能分析データベースです。`StarRocks`は、多次元分析、リアルタイム分析、アドホッククエリを含む全ての分析シナリオに対応する次世代のサブセカンドMPPデータベースです。>> > [StarRocks](https://www.starrocks.io/) is a High-Performance Analytical Database.> > `StarRocks` is a next-gen sub-second MPP database for full analytics scenarios, including multi-dimensional analytics, real-time analytics and ad-hoc query.> `StarRocks`は通常OLAPに分類され、[ClickBench — 分析用DBMSのベンチマーク](https://benchmark.clickhouse.com/)で優れたパフォーマンスを発揮しています。超高速なベクトル化実行エンジンを搭載しているため、高速なベクトルデータベースとしても利用できます。>> > Usually `StarRocks` is categorized into OLAP, and it has showed excellent performance in [ClickBench — a Benchmark For Analytical DBMS](https://benchmark.clickhouse.com/). Since it has a super-fast vectorized execution engine, it could also be used as a fast vectordb.ここでは、StarRocks Vector Storeの使用方法をご紹介します。> Here we'll show how to use the StarRocks Vector Store.

## Setup | セットアップ

In [1]:
#!pip install pymysql

最初に `update_vectordb = False` を設定してください。ドキュメントが更新されていない場合、ドキュメントの埋め込みを再構築する必要はありません。> Set `update_vectordb = False` at the beginning. If there is no docs updated, then we don't need to rebuild the embeddings of docs

In [1]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import StarRocks
from langchain.vectorstores.starrocks import StarRocksSettings

update_vectordb = False

/Users/dirlt/utils/py3env/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## Load docs and split them into tokens | ドキュメントを読み込んでトークンに分割する

`docs`ディレクトリ下のすべてのマークダウンファイルを読み込む> Load all markdown files under the `docs` directoryStarRocksのドキュメントについては、https://github.com/StarRocks/starrocks からリポジトリをクローンでき、その中に`docs`ディレクトリがあります。> for starrocks documents, you can clone repo from https://github.com/StarRocks/starrocks, and there is `docs` directory in it.

In [2]:
loader = DirectoryLoader(
    "./docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader
)
documents = loader.load()

ドキュメントをトークンに分割し、新しいドキュメントやトークンがあるため `update_vectordb = True` に設定してください。> Split docs into tokens, and set `update_vectordb = True` because there are new docs/tokens.

In [3]:
# load text splitter and split docs into snippets of text
text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# tell vectordb to update text embeddings
update_vectordb = True

In [4]:
split_docs[-20]

Document(page_content='Compile StarRocks with Docker\n\nThis topic describes how to compile StarRocks using Docker.\n\nOverview\n\nStarRocks provides development environment images for both Ubuntu 22.04 and CentOS 7.9. With the image, you can launch a Docker container and compile StarRocks in the container.\n\nStarRocks version and DEV ENV image\n\nDifferent branches of StarRocks correspond to different development environment images provided on StarRocks Docker Hub.\n\nFor Ubuntu 22.04:\n\n| Branch name | Image name              |\n  | --------------- | ----------------------------------- |\n  | main            | starrocks/dev-env-ubuntu:latest     |\n  | branch-3.0      | starrocks/dev-env-ubuntu:3.0-latest |\n  | branch-2.5      | starrocks/dev-env-ubuntu:2.5-latest |\n\nFor CentOS 7.9:\n\n| Branch name | Image name                       |\n  | --------------- | ------------------------------------ |\n  | main            | starrocks/dev-env-centos7:latest     |\n  | branch-3.0      

In [5]:
print("# docs  = %d, # splits = %d" % (len(documents), len(split_docs)))

# docs  = 657, # splits = 2802


## Create vectordb instance | vectordbインスタンスを作成する

### Use StarRocks as vectordb | StarRocksをvectordbとして使用する

In [6]:
def gen_starrocks(update_vectordb, embeddings, settings):
    if update_vectordb:
        docsearch = StarRocks.from_documents(split_docs, embeddings, config=settings)
    else:
        docsearch = StarRocks(embeddings, settings)
    return docsearch

## Convert tokens into embeddings and put them into vectordb | トークンを埋め込みに変換し、vectordbに格納してください

ここではStarRocksをベクターデータベースとして使用しており、`StarRocksSettings`を通じてStarRocksインスタンスを構成できます。> Here we use StarRocks as vectordb, you can configure StarRocks instance via `StarRocksSettings`.StarRocksインスタンスの設定は、MySQLインスタンスの設定とほぼ同じです。以下を指定する必要があります：> Configuring StarRocks instance is pretty much like configuring mysql instance. You need to specify:1. ホスト/ポート   > host/port2. ユーザー名（デフォルト：'root'）   > username(default: 'root')3. パスワード（デフォルト：''）   > password(default: '')4. データベース（デフォルト：'default'）   > database(default: 'default')5. テーブル(デフォルト: 'langchain')   > table(default: 'langchain')

In [8]:
embeddings = OpenAIEmbeddings()

# configure starrocks settings(host/port/user/pw/db)
settings = StarRocksSettings()
settings.port = 41003
settings.host = "127.0.0.1"
settings.username = "root"
settings.password = ""
settings.database = "zya"
docsearch = gen_starrocks(update_vectordb, embeddings, settings)

print(docsearch)

update_vectordb = False

Inserting data...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2802/2802 [02:26<00:00, 19.11it/s]


zya.langchain @ 127.0.0.1:41003

username: root

Table Schema:
----------------------------------------------------------------------------
|name                    |type                    |key                     |
----------------------------------------------------------------------------
|id                      |varchar(65533)          |true                    |
|document                |varchar(65533)          |false                   |
|embedding               |array<float>            |false                   |
|metadata                |varchar(65533)          |false                   |
----------------------------------------------------------------------------



## Build QA and ask question to it | QAを構築し、それに質問を投げかける

In [10]:
llm = OpenAI()
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)
query = "is profile enabled by default? if not, how to enable profile?"
resp = qa.run(query)
print(resp)

 No, profile is not enabled by default. To enable profile, set the variable `enable_profile` to `true` using the command `set enable_profile = true;`
